<a href="https://colab.research.google.com/github/blewis219/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/BRADLY_LEWIS_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [190]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [191]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# Let's start by getting some general information from our dataframe.

In [192]:
print(df.shape)
df.head()

(48817, 34)


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [193]:
# List the columns alphabetically
list(df.columns.sort_values().values)

['balcony',
 'bathrooms',
 'bedrooms',
 'cats_allowed',
 'common_outdoor_space',
 'created',
 'description',
 'dining_room',
 'dishwasher',
 'display_address',
 'dogs_allowed',
 'doorman',
 'elevator',
 'exclusive',
 'fitness_center',
 'garden_patio',
 'hardwood_floors',
 'high_speed_internet',
 'interest_level',
 'latitude',
 'laundry_in_building',
 'laundry_in_unit',
 'loft',
 'longitude',
 'new_construction',
 'no_fee',
 'outdoor_space',
 'pre-war',
 'price',
 'roof_deck',
 'street_address',
 'swimming_pool',
 'terrace',
 'wheelchair_access']

In [194]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bathrooms             48817 non-null  float64
 1   bedrooms              48817 non-null  int64  
 2   created               48817 non-null  object 
 3   description           47392 non-null  object 
 4   display_address       48684 non-null  object 
 5   latitude              48817 non-null  float64
 6   longitude             48817 non-null  float64
 7   price                 48817 non-null  int64  
 8   street_address        48807 non-null  object 
 9   interest_level        48817 non-null  object 
 10  elevator              48817 non-null  int64  
 11  cats_allowed          48817 non-null  int64  
 12  hardwood_floors       48817 non-null  int64  
 13  dogs_allowed          48817 non-null  int64  
 14  doorman               48817 non-null  int64  
 15  dishwasher         

# We are going to start by **engineering some features.**


In [195]:
# Engineer a feature for perks
df['perks'] = (
    df.elevator +
    df.hardwood_floors +
    df.doorman +
    df.dishwasher +
    df.no_fee +
    df['pre-war'] +
    df.fitness_center +
    df.laundry_in_building +
    df.laundry_in_unit +
    df.roof_deck +
    df.outdoor_space +
    df.dining_room +
    df.high_speed_internet +
    df.balcony +
    df.swimming_pool +
    df.new_construction +
    df.terrace +
    df.exclusive +
    df.loft +
    df.garden_patio +
    df.wheelchair_access +
    df.common_outdoor_space +
    df.cats_allowed +
    df.dogs_allowed
)

# Engineer a feauture for beds_baths_ratio
df['beds_baths_ratio'] = (
    df.bedrooms / df.bathrooms
)

# Engineer a feature for description_len
df['description_len'] = df['description'].str.len()

# Convert 'interest_level' to int values
df.loc[(df['interest_level'] == "low"), 'interest_level'] = 1
df.loc[(df['interest_level'] == "medium"), 'interest_level'] = 2
df.loc[(df['interest_level'] == "high"), 'interest_level'] = 3

In the process of engineering some features, we created some Null values. This was due to the fact that some descriptions had missing data and 0/0 is Not a Real Number. We will replace them with '0' since they are still useable.

In [196]:
# Replace NaN values with 0
df['beds_baths_ratio'] = df['beds_baths_ratio'].fillna(0)
df['description_len'] = df['description_len'].fillna(0)

df['beds_baths_ratio'] = df['beds_baths_ratio'].round(decimals=2)
df = df.replace([np.inf],0)

In [197]:
# Creating a subset of the columns we are interested in
sub_df = df[['bathrooms', 'bedrooms', 'beds_baths_ratio', 'perks', 'description_len', 'interest_level', 'latitude', 'longitude', 'price', 'interest_level', 'cats_allowed', 'dogs_allowed']]

# Splitting the Data into training and testing groups.

In [198]:
# Convert 'created' to a DateTime format so we can separate the month
sub_df['created'] = pd.to_datetime(df['created'])
sub_df['month'] = pd.DatetimeIndex(df['created']).month

# Divide the dataframe into Train and Test
train = sub_df[(sub_df.month == 4) | (sub_df.month == 5)]
test = sub_df[sub_df.month == 6]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# **Linear Regression Model**
We will make a few models to see what gives us the best results.

# Model 1: Bedrooms & Bathrooms

In [199]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrices and y target vectors
features = ['bedrooms', 'bathrooms']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:,.2f}')

Train Error: $818.53


In [200]:
# 5. Apply the model to new data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:,.2f}')

Test Error: $825.90


In [201]:
# Model's coeficient and intercept
coef1 = model.coef_[0]
coef2 = model.coef_[1]
intercept = model.intercept_
print(f'The coeficients are {coef1:,.2f} and {coef2:,.2f}. The intercept is {intercept:,.2f}.')

The coeficients are 389.32 and 2,072.61. The intercept is 485.72.


In [202]:
# R^2 values for the training data and testing data
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)

print(f'The Training model R-squared value is {r2_train:,.2f}')
print(f'The Testing model R-squared value is {r2_test:,.2f}')

The Training model R-squared value is 0.51
The Testing model R-squared value is 0.52


**I did add in bed_bath_ratio but it caused my error to go up and r^2 to go down, so ultimately left it out.**

# Model 2: Bedrooms, Bathrooms, and Perks

In [203]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrices and y target vectors
features = ['bedrooms', 'bathrooms', 'perks']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:,.2f}')

Train Error: $795.41


In [204]:
# 5. Apply the model to new data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:,.2f}')

Test Error: $799.74


In [205]:
# Model's coeficient and intercept
coef1 = model.coef_[0]
coef2 = model.coef_[1]
coef3 = model.coef_[2]

intercept = model.intercept_
print(f'The coeficients are {coef1:,.2f}, {coef2:,.2f} and {coef3:,.2f}. The intercept is {intercept:,.2f}.')

The coeficients are 391.99, 1,936.28 and 78.17. The intercept is 277.61.


In [206]:
# R^2 values for the training data and testing data
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)

print(f'The Training model R-squared value is {r2_train:,.2f}')
print(f'The Testing model R-squared value is {r2_test:,.2f}')

The Training model R-squared value is 0.53
The Testing model R-squared value is 0.54


# Model 3: Bedrooms, Bathrooms, Perks, and Description Len

In [207]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrices and y target vectors
features = ['bedrooms', 'bathrooms', 'perks', 'description_len']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:,.2f}')

Train Error: $795.39


In [208]:
# 5. Apply the model to new data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:,.2f}')

Test Error: $799.72


In [209]:
# Model's coeficient and intercept
coef1 = model.coef_[0]
coef2 = model.coef_[1]
coef3 = model.coef_[2]
coef4 = model.coef_[3]

intercept = model.intercept_
print(f'The coeficients are {coef1:,.2f}, {coef2:,.2f}, {coef3:,.2f} and {coef4:,.2f}. The intercept is {intercept:,.2f}.')

The coeficients are 392.08, 1,936.35, 78.44 and -0.01. The intercept is 279.62.


In [210]:
# R^2 values for the training data and testing data
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)

print(f'The Training model R-squared value is {r2_train:,.2f}')
print(f'The Testing model R-squared value is {r2_test:,.2f}')

The Training model R-squared value is 0.53
The Testing model R-squared value is 0.54


# Model 4: Bedrooms, Bathrooms, Perks, Description_Len, Lat, long, and Interest_Level

In [211]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

# 2. Instantiate this class
model = LinearRegression()

# 3. Arrange X features matrices and y target vectors
features = ['bedrooms', 'bathrooms', 'perks', 'description_len', 'latitude', 'longitude', 'interest_level']
target = 'price'

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

# 4. Fit the model
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:,.2f}')

Train Error: $700.53


In [212]:
# 5. Apply the model to new data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:,.2f}')

Test Error: $701.73


In [213]:
# Model's coeficient and intercept
coef1 = model.coef_[0]
coef2 = model.coef_[1]
coef3 = model.coef_[2]
coef4 = model.coef_[3]
coef5 = model.coef_[4]
coef6 = model.coef_[5]
coef7 = model.coef_[6]
intercept = model.intercept_
print(f'The coeficients are {coef1:,.2f}, {coef2:,.2f}, {coef3:,.2f}, {coef4:,.2f}, {coef5:,.2f}, {coef6:,.2f} and {coef7:,.2f}. The intercept is {intercept:,.2f}.')

The coeficients are 451.87, 1,839.25, 51.21, 0.05, 1,372.63, -14,047.04 and -236.01. The intercept is -1,093,980.50.


In [214]:
# R^2 values for the training data and testing data
r2_train = model.score(X_train, y_train)
r2_test = model.score(X_test, y_test)

print(f'The Training model R-squared value is {r2_train:,.2f}')
print(f'The Testing model R-squared value is {r2_test:,.2f}')

The Training model R-squared value is 0.61
The Testing model R-squared value is 0.62


---

# Results:


*   We tried using the model on 4 different sets of features
*   The features that worked the best are **Bedrooms, Bathrooms, Perks, Description_Len, Lat, long, and Interest_Level**
*   Our MAE is Train: 700.53 | Test: 701.73
*   The R-squared values are Train: 0.61 | Test: 0.62



